# 一、导入需要使用的库

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os,sys
import langid
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import importlib
import numpy as np

def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal, LAParams
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfpage import PDFPage

### step2-2:从sci-hub上批量下载所有需要的文献

In [19]:
#设定下载地址
options=webdriver.ChromeOptions()
out_path="/Users/yanglulin/Documents/本科学习生活/三春夏/4-计算文本分析/考核相关/大作业相关/PaperDownload/papers"
prefs={'profile.default_content_settings.popups': 0, 'download.default_directory': out_path}
options.add_experimental_option('prefs', prefs)

In [5]:
#为了帮助判断是否下载成功，定义一个计算文件夹中的文献数的函数
def count_paper(path):
    for root,dirs,files in os.walk(path):
        file_num=len(files)-1
    return (file_num)

In [21]:
#定义一个函数，帮助找到最新下载的一个文件
def find_new_file(path):
    file_lists=os.listdir(path)
    file_lists.remove(".DS_Store")
    file_lists.sort(key=lambda fn: os.path.getmtime(path + "/" + fn)
                    if not os.path.isdir(path + "/" + fn) else 0)
    file=os.path.join(path, file_lists[-1])
    return file

In [48]:
df=links1940_1945
print(len(df))

256


In [49]:
#根据已经获取的stable_url在sci-hub上批量下载文献

label=True #作用是当确定所有文献都已经下载完之后，就使循环停止运行
times=0 #记录整体循环的次数；有些没有一次就下载成功的文章需要多次尝试，通过给times设定上限确保程序最终会停止
count=count_paper(out_path)

while label==True and times<5:
    times+=1
    print("循环次数：",times)
    unsuc=0 #判断是否有未下载成功的文献

    for i in range(len(df)):
        flushPrint(i+1)
       
        if df.iloc[i,2]==0:
            
            try:
                
                driver=webdriver.Chrome(executable_path="/Applications/anaconda3/chromedriver",chrome_options=options)
                link="https://sci-hub.hkvisa.net"+"/"+df.iloc[i,1]
                driver.get(link)

                soup=BeautifulSoup(driver.page_source,'lxml')
                download_link=str(soup.select("#buttons>button"))
                download_link=download_link.split("href='")
                download_link=download_link[1].split("'\">")
                download_link=download_link[0]


                driver.get(download_link)
                time.sleep(8)
                
                #判断文献是否下载成功
                if count_paper(out_path)==count+1:
                    df.iloc[i,2]=1
                    count+=1
                    
                    #给新下载的文件重命名
                    new_paper=find_new_file(out_path)
                    new_name=str(df.iloc[i,0])+"-"+df.iloc[i,1][-7:]
                    os.rename(new_paper,out_path+"/"+new_name+".pdf")
                    
                
            except:
                unsuc+=1
            
            driver.quit()
        
    print()
    
    if unsuc==0:
        label=False

循环次数： 1
256
循环次数： 2
256
循环次数： 3
256
循环次数： 4
256
循环次数： 5
256


In [50]:
#统计下载成功的文献数
count_paper(out_path)

3208

In [51]:
#查看没有下载成功的文献，尝试手动下载补齐
count=0
for i in range(len(df)):
    if df.iloc[i,2]==0:
        count+=1
        print(df.iloc[i,1])
print(count)

https://www.jstor.org/stable/2181134
https://www.jstor.org/stable/2180964
https://www.jstor.org/stable/2180560
https://www.jstor.org/stable/2181365
https://www.jstor.org/stable/2181354
5


### step2-3:把下载好的pdf文件转成txt格式

In [2]:
importlib.reload(sys)

#定义一个可以实现pdf转txt的函数
def parse(pdf_path, txt_path,unwanted):
    label=True
    
    # 解析PDF文本，并保存到TXT文件中
    fp = open(pdf_path, 'rb')
    # 用文件对象创建一个PDF文档分析器
    parser = PDFParser(fp)
    # 创建一个PDF文档
    doc = PDFDocument(parser,'')
    # 连接分析器，与文档对象
    parser.set_document(doc)
    
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        # 创建PDF，资源管理器，来共享资源
        rsrcmgr = PDFResourceManager()
        # 创建一个PDF设备对象
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        # 创建一个PDF解释器对象
        interpreter = PDFPageInterpreter(rsrcmgr, device)

        # 循环遍历列表，每次处理一个page内容
        pages=list(PDFPage.create_pages(doc))
        page_num=len(pages)
        i=0
        while i<page_num and label:
            page=pages[i]
            interpreter.process_page(page)
            # 接受该页面的LTPage对象
            layout = device.get_result()
            # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象
            # 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等
            # 想要获取文本就获得对象的text属性
            for x in layout:
                if (isinstance(x, LTTextBoxHorizontal)):
                    results = x.get_text()
                    if i==0:
                        for t in unwanted:
                            if t in results:
                                label=False
                                break
                    else:
                        with open(txt_path, 'a+', encoding='utf-8') as f:
                            f.write(results + "\n")
            i+=1

In [8]:
#对所有下载得到的pdf文件使用parse函数
pdf_path="/Users/yanglulin/Documents/本科学习生活/三春夏/4-计算文本分析/考核相关/大作业相关/PaperDownload/papers"
txt_path="/Users/yanglulin/Documents/本科学习生活/三春夏/4-计算文本分析/考核相关/大作业相关/PaperDownload/papers_txt"
unwanted=["Volume Information","Front Matter","Books Received","Other Books Received","Descriptive Notices","Notes","Back Matter"]

pdf_docs=[f for f in os.listdir(pdf_path)]
txt_docs=[f for f in os.listdir(txt_path)]
pdf_docs.remove(".DS_Store")

unsuc=0

for f in pdf_docs:
    txt_name=f[:-3]+"txt"
    if txt_name not in txt_docs:
        try:
            parse(pdf_path+"/"+f, txt_path+"/"+txt_name,unwanted)
        except:
            print(f)
            unsuc+=1

In [9]:
#统计转化后的文献数
count_paper(txt_path)

1748

In [10]:
#其中转换失败的数量是：
unsuc

0